In [ ]:
# ✅ CNN Fake News Classification Using ResNet-50 (Image Dataset)

# ✅ Install Required Libraries (only run once)
!pip install torch torchvision

import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import shutil
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ✅ Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# ✅ Dataset Directory Setup
data_dir = "/content/drive/MyDrive/thesis/Dataset"
batch_size = 16
num_classes = 2  # Real vs Fake


In [ ]:
# ✅ Data Transforms for Each Phase
data_transforms = {
    "Train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "Validation": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    "Test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}




In [ ]:
# ✅ Load Datasets
image_datasets = {
    phase: datasets.ImageFolder(os.path.join(data_dir, phase), data_transforms[phase])
    for phase in ['Train', 'Validation', 'Test']
}

In [ ]:
dataloaders = {
    phase: DataLoader(image_datasets[phase], batch_size=batch_size, shuffle=True, num_workers=2)
    for phase in ['Train', 'Validation', 'Test']
}

In [ ]:
# ✅ Load Pretrained ResNet50 and Modify Classifier
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 79.6MB/s]


In [ ]:
# ✅ Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# ✅ Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloaders['Train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")


Epoch 1/5
Train Loss: 0.2901, Accuracy: 0.9213
Epoch 2/5
Train Loss: 0.1558, Accuracy: 0.9364
Epoch 3/5
Train Loss: 0.0872, Accuracy: 0.9696
Epoch 4/5
Train Loss: 0.0552, Accuracy: 0.9839
Epoch 5/5
Train Loss: 0.0512, Accuracy: 0.9810


In [ ]:
!pip install scikit-learn numpy

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# ✅ Evaluation on Test Set with detailed metrics
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in dataloaders['Test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ✅ Class names (make sure your folder names in Test set match this order)
class_names = image_datasets['Test'].classes  # ['Fake', 'Real'] or ['Real', 'Fake']

# ✅ Print classification report
print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))

# ✅ Optional: Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Classification Report:

              precision    recall  f1-score   support

        Fake       0.79      0.44      0.56       227
        Real       0.60      0.88      0.71       218

    accuracy                           0.65       445
   macro avg       0.70      0.66      0.64       445
weighted avg       0.70      0.65      0.64       445

Confusion Matrix:
[[ 99 128]
 [ 26 192]]
